In [ ]:
# ✅ Step 1: Install dependencies
!pip install -q transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00


In [ ]:
# ✅ Step 2: Check GPU availability
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from huggingface_hub import login

# Paste your token between the quotes
login("Your Access Token")


In [ ]:
# ✅ Step 3: Load tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA needs a pad token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
!rm -rf ~/.cache/huggingface/datasets


In [ ]:
import json
from datasets import Dataset

# Load JSON file manually
with open("/content/medical_dataset_llama3_format.json") as f:
    try:
        data=json.load(f)
        print("JSON is valid!")
    except json.JSONDecodeError as e:
        print(f"Invalid JSON: {e}")


# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)

# Train/test split
dataset = dataset.train_test_split(test_size=0.2)

# View a sample
print(dataset["train"][0])


JSON is valid!
{'messages': [{'content': 'You are a helpful medical AI assistant specialized in providing accurate medical information and advice.', 'role': 'system'}, {'content': '<|start_header_id|>user<|end_header_id|>\n\nI feel disconnected from every one and everything in my life. I feel overwhelmed all time, I cannot sleepwell. I have racing thoughts. If I fall asleeep I awake with a start in an hour or so. I am always hungry. I am experiencing pain in several joints. I am diabetic and my sugars have been around 200 whether I eat or not.<|eot_id|>', 'role': 'user'}, {'content': '<|start_header_id|>assistant<|end_header_id|>\n\nHello and Welcome to  \x80\x98Ask A Doctor \x80\x99 service. I have reviewed your query and here is my advice. Anxiety and body pains can co-occur with depression. As depression will improve these symptoms will subside.Depression can affect anyone and you can overcome it completely. Depression symptoms include feeling low all the time, loss of interest in p

In [ ]:


def safe_convert_to_messages(data):
    """Safely convert message data to proper format"""
    if isinstance(data, str):
        try:
            return json.loads(data.replace("'", '"'))
        except json.JSONDecodeError:
            return [{"role": "user", "content": data}]
    elif isinstance(data, list):
        return data
    return [{"role": "user", "content": str(data)}]

def format_conversation(messages):
    """Convert messages to Llama 3 chat format"""
    formatted = []
    for msg in messages:
        if not isinstance(msg, dict):
            msg = {"role": "user", "content": str(msg)}
        content = msg.get("content", "")
        # Clean special tokens if present
        if "<|start_header_id|>" in content:
            content = content.split("\n\n")[-1].replace("<|eot_id|>", "")
        formatted.append({"role": msg.get("role", "user"), "content": content})
    return formatted

def tokenize_function(batch):
    """Process batch of examples"""
    formatted_texts = []
    for messages in batch["messages"]:
        # Convert to proper message format
        messages = safe_convert_to_messages(messages)
        messages = format_conversation(messages)

        # Apply chat template
        try:
            formatted = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            formatted_texts.append(formatted)
        except Exception as e:
            print(f"Skipping malformed message: {e}")
            formatted_texts.append("")  # Empty string for padding

    # Tokenize all texts
    tokenized = tokenizer(
        formatted_texts,
        truncation=True,
        padding="max_length",
        max_length=1024,  # Reduced for safety
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

# Apply tokenization
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=["messages"]
)

# Set format for training
tokenized_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/217752 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Skipping malformed message: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating
Skipping malformed message: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating
Skipping malformed message: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.c

Map:   0%|          | 0/54439 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Skipping malformed message: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating
Skipping malformed message: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating
Skipping malformed message: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.c

In [ ]:
# ✅ Step 5: Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenizer to both train and test sets
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Format for PyTorch
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/217752 [00:00<?, ? examples/s]

KeyError: 'text'

In [ ]:
from torch.utils.data import DataLoader

# Correct way to split the dataset (using the dataset's method)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Then create DataLoaders
train_dataloader = DataLoader(
    split_dataset["train"],
    batch_size=2,
    shuffle=True
)

eval_dataloader = DataLoader(
    split_dataset["test"],
    batch_size=2
)

print(f"Train batches: {len(train_dataloader)}")
print(f"Eval batches: {len(eval_dataloader)}")

NameError: name 'tokenized_dataset' is not defined

In [ ]:
from torch.optim import AdamW

# Initialize optimizer with model parameters and learning rate
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)


In [ ]:
!pip install numpy==1.26.4
import os
os._exit(00)  # Restart the runtime after install


In [ ]:
from tqdm import tqdm
import torch

model.train()
model.to(device)

epochs = 1

for epoch in range(epochs):
    loop = tqdm(train_dataloader, leave=True)

    for batch in loop:
        # Move inputs to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Forward pass with labels = input_ids for causal language modeling
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Logging
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())


Epoch 1:  35%|███▌      | 38171/108876 [3:22:20<6:14:50,  3.14it/s, loss=nan]

In [ ]:
# ✅ Step 9: Save model and tokenizer
model.save_pretrained("./llama3-1b-finetuned-manual")
tokenizer.save_pretrained("./llama3-1b-finetuned-manual")